# What ETFs should I get for long term investing?

In [ ]:
# Important Links 

# Documentations
#   https://pypi.org/project/yfinance/
#   https://github.com/ranaroussi/yfinance
# ETF
#   https://etfdb.com/

# https://www.kaggle.com/code/maisonr/horror-movies

In [1]:
import yfinance as yf
import pandas as pd
import os

#### Getting a list of etfs that I have seen on investing / finance subreddits, Youtube, a general investing articles

In [109]:
tickersList = ['VOO', 'IVV', 'SPY', 'QQQ', 'SPLG', 'VTI', 'VT']

#### Turning CSV's into 2 dataframes, 1 for ETF info 1 for ETF stock weights

In [103]:
weightDF = []
infoDF = []

for ticker in tickersList:
    # first 8 rows of each csv to turn into an info df
    tickerInfo = pd.read_csv(f'{ticker}-holdings.csv', delimiter= ':', nrows= 8, header= None)
    
    # transposing or pivoting df
    tickerInfo = tickerInfo.T
    
    # cleaning the df
    tickerInfo.columns = tickerInfo.loc[0]
    tickerInfo = tickerInfo.iloc[1:]
    tickerInfo.reset_index(drop=True, inplace=True)
    tickerInfo = tickerInfo[['Inception Date', 'Total Assets Under Management (in thousands)', 'Expense Ratio', 'Tracks This Index']]
    
    # getting the name of the file / etf 
    name = os.path.basename(f'{ticker}-holdings.csv')
    name = name.split('-')[0]
    
    tickerInfo.loc[:, 'etf'] = name
    
    # moving the column so that etf is first
    tickerInfo = tickerInfo[['etf'] + [col for col in tickerInfo.columns if col != 'etf']]
    
    # getting rid of white space 
    tickerInfo = tickerInfo.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    # finally appending all the df's into one
    infoDF.append(tickerInfo)
    
    
    
    # stocks weight df
    tickerWeight = pd.read_csv(f'{ticker}-holdings.csv', skiprows= 11)
    tickerWeight['etf'] = name
    tickerWeight = tickerWeight[['etf'] + [col for col in tickerWeight.columns if col != 'etf']]
    
    weightDF.append(tickerWeight)
    
infoDF = pd.concat(infoDF, ignore_index= True)
weightDF = pd.concat(weightDF, ignore_index= True)

In [105]:
# making sure I got all the ETFs
print(weightDF['etf'].unique())
print(infoDF['etf'].unique())

['VOO' 'IVV' 'SPY' 'QQQ' 'SPLG' 'VTI' 'VT']
['VOO' 'IVV' 'SPY' 'QQQ' 'SPLG' 'VTI' 'VT']


In [106]:
infoDF.to_csv('etf_info.csv', index= False)
weightDF.to_csv('etf_weights.csv', index= False)

#### Getting historical data of ETFs

In [143]:
tickersList = ['VOO', 'IVV', 'SPY', 'QQQ', 'SPLG', 'VTI', 'VT']
historicalDF = []

for ticker in tickersList:    
    ticker = yf.Ticker(f'{ticker}')
    df = ticker.history(period= 'max')
    df.reset_index(inplace= True)
    df['etf'] = ticker
    df = df[['etf'] + [col for col in df.columns if col != 'etf']]
    df.drop(['Open', 'High', 'Low', 'Stock Splits', 'Capital Gains'], axis = 1, inplace= True)
    historicalDF.append(df)
    
    
historicalDF = pd.concat(historicalDF, ignore_index= True)

In [145]:
historicalDF.to_csv('etf_historical.csv', index= False)

#### Getting top 20 weighted stocks' historical data and all the industries from distinct companies

Stock Historical Data

In [173]:
top20 = pd.read_csv('top 20 stock weights.csv')
top20List = top20['Symbol'].tolist()

In [180]:
symbol20 = []

for symbol in top20List:    
    symbol = yf.Ticker(f'{symbol}')
    df = symbol.history(period= 'max')
    df.reset_index(inplace= True)
    df['symbol'] = symbol
    df = df[['symbol'] + [col for col in df.columns if col != 'symbol']]
    symbol20.append(df)
    
    
symbol20 = pd.concat(symbol20, ignore_index= True)
symbol20 = symbol20[['symbol', 'Date', 'Close', 'Volume']]

symbol20.to_csv('stock20_historical.csv', index= False)

$BRK.B: possibly delisted; No timezone found
/var/folders/hk/jzskg0xs5pjggw047qcwrp980000gn/T/ipykernel_2329/1976440890.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  symbol20 = pd.concat(symbol20, ignore_index= True)


Stock Industry Data

In [15]:
stocks = pd.read_csv('all available stocks.csv')
stocksList = stocks['symbol'].tolist()

In [16]:
print(len(stocksList))

5518


In [17]:
symbolDF = []

for symbol in stocksList:    
    symbol = yf.Ticker(f'{symbol}')
    dict = symbol.info
    df = pd.DataFrame([dict])
    df['symbol'] = symbol
    symbolDF.append(df)
    
    
symbolDF = pd.concat(symbolDF, ignore_index= True)
symbolDF = symbolDF[['symbol', 'industry', 'sector']]
symbolDF.drop_duplicates(inplace= True)
symbolDF.dropna(inplace= True)

symbolDF.to_csv('stock_industry.csv', index= False)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DLEA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DLEA&crumb=M8f%2FvlqITQi
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BIMBOA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BIMBOA&crumb=M8f%2FvlqITQi
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/EMPOWER?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=EMPOWER&crumb=M8f%2FvlqITQi
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BCHN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDom

In [18]:
symbolDF.shape[0]

3981